# 05. 3D音響可視化

音響データを3次元で可視化して、より直感的に音の特性を理解します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 3D波形表示

In [ ]:
# 複数の周波数を持つ信号の3D表示
def create_3d_waveform():
    sample_rate = 8000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 異なる周波数の信号を生成
    frequencies = [200, 400, 600, 800, 1000]
    signals = []
    
    for freq in frequencies:
        amplitude = 1.0 / (freq / 200)  # 高周波数ほど小さい振幅
        wave = amplitude * np.sin(2 * np.pi * freq * t)
        signals.append(wave)
    
    return t, frequencies, signals

t, frequencies, signals = create_3d_waveform()

# 3D表示
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

# 各周波数の波形を3D空間に配置
colors = plt.cm.viridis(np.linspace(0, 1, len(frequencies)))

for i, (freq, wave, color) in enumerate(zip(frequencies, signals, colors)):
    # 時間軸の一部のみ表示（計算量削減）
    step = 50  # サンプリング間隔
    t_plot = t[::step]
    wave_plot = wave[::step]
    freq_array = np.full_like(t_plot, freq)
    
    ax.plot(t_plot, freq_array, wave_plot, color=color, 
           linewidth=2, label=f'{freq} Hz')

ax.set_xlabel('時間 (秒)')
ax.set_ylabel('周波数 (Hz)')
ax.set_zlabel('振幅')
ax.set_title('3D波形表示')
ax.legend()

plt.show()

## 2. 3Dスペクトログラム

In [ ]:
# チャープ信号のスペクトログラムを3Dで表示
sample_rate = 4000
duration = 3.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# チャープ信号（周波数が変化）
chirp = signal.chirp(t, 100, duration, 1500, method='linear')

# スペクトログラムの計算
frequencies, times, Sxx = signal.spectrogram(chirp, sample_rate,
                                           window='hann', nperseg=256,
                                           noverlap=128)

# 3D表示用のメッシュグリッドを作成
T, F = np.meshgrid(times, frequencies)

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

# 3Dサーフェス表示
surf = ax.plot_surface(T, F, 10 * np.log10(Sxx + 1e-10), 
                      cmap='plasma', alpha=0.8,
                      linewidth=0, antialiased=True)

ax.set_xlabel('時間 (秒)')
ax.set_ylabel('周波数 (Hz)')
ax.set_zlabel('パワー (dB)')
ax.set_title('3Dスペクトログラム')

# カラーバーを追加
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=20, label='パワー (dB)')

plt.show()

## 3. ウォーターフォール表示

In [ ]:
# 時間的に変化するスペクトルのウォーターフォール表示
def create_evolving_spectrum():
    sample_rate = 4000
    duration = 0.5  # 短い時間窓
    num_spectra = 20  # スペクトル数
    
    spectra = []
    frequencies = None
    
    for i in range(num_spectra):
        t = np.linspace(0, duration, int(sample_rate * duration), False)
        
        # 基本周波数が時間とともに変化
        base_freq = 200 + i * 30  # 200Hzから800Hzまで変化
        
        # 基音と倍音を含む信号
        signal = (np.sin(2 * np.pi * base_freq * t) +
                 0.5 * np.sin(2 * np.pi * 2 * base_freq * t) +
                 0.3 * np.sin(2 * np.pi * 3 * base_freq * t))
        
        # FFTでスペクトルを計算
        fft_result = np.fft.fft(signal)
        if frequencies is None:
            frequencies = np.fft.fftfreq(len(signal), 1/sample_rate)
            # 正の周波数のみ
            positive_freq_mask = frequencies >= 0
            frequencies = frequencies[positive_freq_mask]
        
        spectrum = np.abs(fft_result[positive_freq_mask])
        spectra.append(spectrum)
    
    return frequencies, spectra

frequencies, spectra = create_evolving_spectrum()

# ウォーターフォール表示
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

# 表示範囲を制限（高周波数をカット）
freq_limit = 2000
freq_mask = frequencies <= freq_limit
freq_display = frequencies[freq_mask]

# 各時間ステップでのスペクトルを表示
colors = plt.cm.coolwarm(np.linspace(0, 1, len(spectra)))

for i, (spectrum, color) in enumerate(zip(spectra, colors)):
    spectrum_display = spectrum[freq_mask]
    spectrum_db = 20 * np.log10(spectrum_display + 1e-10)
    
    # Y軸をオフセット（時間軸として使用）
    y_offset = i * 0.5
    
    ax.plot(freq_display, np.full_like(freq_display, y_offset), 
           spectrum_db, color=color, linewidth=2, alpha=0.8)

ax.set_xlabel('周波数 (Hz)')
ax.set_ylabel('時間ステップ')
ax.set_zlabel('振幅 (dB)')
ax.set_title('ウォーターフォール表示（スペクトルの時間変化）')

# 視点を調整
ax.view_init(elev=20, azim=45)

plt.show()

## 4. 円筒座標系での可視化

In [ ]:
# 円筒座標系での音響可視化
def create_cylindrical_visualization():
    sample_rate = 8000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 複数の周波数成分を持つ信号
    frequencies = [200, 400, 600, 800]
    amplitudes = [1.0, 0.7, 0.5, 0.3]
    
    # 合成信号
    composite_signal = np.zeros_like(t)
    for freq, amp in zip(frequencies, amplitudes):
        composite_signal += amp * np.sin(2 * np.pi * freq * t)
    
    return t, composite_signal, frequencies, amplitudes

t, composite_signal, frequencies, amplitudes = create_cylindrical_visualization()

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

# 円筒座標系でのプロット
# 時間を角度に、周波数を半径に、振幅をZ軸にマッピング

# サンプリング間隔を大きくして表示点を減らす
step = 100
t_sample = t[::step]
signal_sample = composite_signal[::step]

# 各周波数成分を個別に表示
colors = plt.cm.rainbow(np.linspace(0, 1, len(frequencies)))

for i, (freq, amp, color) in enumerate(zip(frequencies, amplitudes, colors)):
    # この周波数成分の信号
    freq_signal = amp * np.sin(2 * np.pi * freq * t_sample)
    
    # 円筒座標系での座標計算
    theta = 2 * np.pi * t_sample / max(t_sample)  # 時間を角度に変換
    r = freq / 100  # 周波数を半径に変換（スケーリング）
    z = freq_signal  # 振幅をZ座標に
    
    # 直交座標系に変換
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    ax.plot(x, y, z, color=color, linewidth=2, alpha=0.8, 
           label=f'{freq} Hz')

# 中心軸を表示
z_axis = np.linspace(-1, 1, 50)
ax.plot([0]*len(z_axis), [0]*len(z_axis), z_axis, 'k--', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('振幅')
ax.set_title('円筒座標系での音響可視化\n（半径=周波数, 角度=時間, 高さ=振幅）')
ax.legend()

plt.show()

## 5. インタラクティブな3D可視化

In [ ]:
# 複数の視点から音響データを観察
def create_multi_view_visualization():
    # チャープ信号の生成
    sample_rate = 4000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 周波数が変化するチャープ信号
    chirp = signal.chirp(t, 100, duration, 1000, method='quadratic')
    
    # スペクトログラムの計算
    frequencies, times, Sxx = signal.spectrogram(chirp, sample_rate,
                                               window='hann', nperseg=128,
                                               noverlap=64)
    
    return t, chirp, frequencies, times, Sxx

t, chirp, frequencies, times, Sxx = create_multi_view_visualization()

# 複数の視点での表示
fig = plt.figure(figsize=(20, 15))

# 視点1: 上から見た図（従来のスペクトログラム）
ax1 = fig.add_subplot(2, 3, 1)
ax1.pcolormesh(times, frequencies, 10 * np.log10(Sxx + 1e-10), 
              shading='gouraud', cmap='viridis')
ax1.set_title('上面図（従来のスペクトログラム）')
ax1.set_xlabel('時間 (秒)')
ax1.set_ylabel('周波数 (Hz)')

# 視点2: 横から見た図（時間 vs パワー）
ax2 = fig.add_subplot(2, 3, 2)
total_power = np.sum(Sxx, axis=0)  # 各時刻での総パワー
ax2.plot(times, 10 * np.log10(total_power + 1e-10), 'b-', linewidth=2)
ax2.set_title('側面図（時間 vs 総パワー）')
ax2.set_xlabel('時間 (秒)')
ax2.set_ylabel('総パワー (dB)')
ax2.grid(True)

# 視点3: 正面から見た図（周波数 vs パワー）
ax3 = fig.add_subplot(2, 3, 3)
avg_spectrum = np.mean(Sxx, axis=1)  # 時間平均スペクトル
ax3.plot(10 * np.log10(avg_spectrum + 1e-10), frequencies, 'r-', linewidth=2)
ax3.set_title('正面図（平均スペクトル vs 周波数）')
ax3.set_xlabel('平均パワー (dB)')
ax3.set_ylabel('周波数 (Hz)')
ax3.grid(True)

# 3D表示（異なる視点）
T, F = np.meshgrid(times, frequencies)

# 視点4: 3D - 視点1
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
surf1 = ax4.plot_surface(T, F, 10 * np.log10(Sxx + 1e-10), 
                        cmap='plasma', alpha=0.8)
ax4.view_init(elev=30, azim=45)
ax4.set_title('3D視点1')

# 視点5: 3D - 視点2
ax5 = fig.add_subplot(2, 3, 5, projection='3d')
surf2 = ax5.plot_surface(T, F, 10 * np.log10(Sxx + 1e-10), 
                        cmap='plasma', alpha=0.8)
ax5.view_init(elev=60, azim=135)
ax5.set_title('3D視点2')

# 視点6: 3D - 視点3
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
surf3 = ax6.plot_surface(T, F, 10 * np.log10(Sxx + 1e-10), 
                        cmap='plasma', alpha=0.8)
ax6.view_init(elev=10, azim=90)
ax6.set_title('3D視点3')

plt.tight_layout()
plt.show()

# 元の信号も表示
plt.figure(figsize=(12, 4))
plt.plot(t, chirp)
plt.title('元のチャープ信号')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)
plt.show()

## 練習問題

1. 楽器の倍音構造を3Dで可視化してみましょう（基音と複数の倍音）
2. ドップラー効果をシミュレートして3Dで表示してみましょう
3. 複数の楽器が演奏するアンサンブルを3D空間で表現してみましょう